# task 1 
get the unigrams and bigrams

In [1]:

from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import word_tokenize
from nltk import FreqDist
    
    
def filter_invalid_ngrams(nested_list):
    """
    Filters a nested list of n-gram tuples to remove invalid combinations.

    An n-gram is considered invalid if it contains consecutive '<s>' or '</s>' tokens.
    For example: ('<s>', '<s>'), ('<s>', '<s>', 'i'), ('sam', '</s>', '</s>')

    Args:
        nested_list: A list of lists, where each inner list contains tuples of strings.

    Returns:
        A new nested list with the invalid n-grams removed.
    """
    filtered_nested_list = []
    for inner_list in nested_list:
        filtered_inner_list = []
        for ngram_tuple in inner_list:
            is_valid = True
            # Check for consecutive <s> or </s> tokens
            for i in range(len(ngram_tuple) - 1):
                if (ngram_tuple[i] == '<s>' and ngram_tuple[i+1] == '<s>') or \
                   (ngram_tuple[i] == '</s>' and ngram_tuple[i+1] == '</s>'):
                    is_valid = False
                    break
            
            if is_valid:
                filtered_inner_list.append(ngram_tuple)
        
        filtered_nested_list.append(filtered_inner_list)
        
    return filtered_nested_list

def TextFile_Corpus(path = "./corpus.txt"):
    
    """
    Reads a text file and prepares a corpus by converting all text to lowercase 
    and stripping any leading or trailing whitespace from each line.

    This function takes the path to a text file as input, reads its contents, 
    and processes each line to create a list of sentences. Each sentence is 
    converted to lowercase to ensure uniformity, making it easier to analyze 
    the text later on.

    Parameters:
    ----------
    path : str, optional
        The file path to the text file containing the corpus. The default value 
        is "./corpus.txt", which means it will look for a file named 'corpus.txt' 
        in the current directory. You can specify a different path if your file 
        is located elsewhere.

    Returns:
    -------
    corpus : list of str
        A list of strings, where each string represents a line from the text file. 
        Each line is processed to be in lowercase and stripped of any extra spaces. 
        This list serves as the basis for further text analysis or natural language 
        processing tasks.
        
    """
    with open(path, 'r', encoding='utf-8') as f:
        corpus = [s.strip().lower() for s in f.readlines()]
    return corpus


corpus = TextFile_Corpus(path="./corpus.txt")

def get_Ngrams(train_data, n):
    Ngrams = [[gram for gram in train_data[i] if len(gram) == n] for i in range(len(train_data)) ]
    return Ngrams

def prepare_corupus (corpus, n = 1):
    
    """
    Prepares a text corpus for n-gram modeling by tokenizing the sentences, 
    generating n-grams, and organizing them into various structures.

    This function takes a list of sentences (the corpus) and processes it to 
    create n-grams of specified lengths. It tokenizes the sentences, converts 
    them to lowercase, and generates padded n-grams. The output includes unique 
    n-grams, flattened n-grams, and the original n-grams in their nested form, 
    along with the training data and vocabulary.

    Parameters:
    ----------
    corpus : list of str
        A list of sentences (strings) that make up the text corpus. Each sentence 
        should be a coherent string of words. For example:
        [
            "I am Sam",
            "Sam I am",
            "Sam I like",
            "I do like Sam",
            "do I like Sam"
        ]

    n : int, optional
        The maximum length of n-grams to generate. The default is 1, which means 
        the function will generate unigrams. If set to 2, it will generate 
        bigrams, and so on. This allows for flexibility in the type of n-grams 
        you want to analyze.

    Returns:
    -------
    unique_n_grams : dict
        A dictionary where each key corresponds to the n-gram length (e.g., 
        '1-grams', '2-grams') and the value is a list of unique n-grams of that 
        length found in the corpus. This helps in understanding the distinct 
        phrases present in the text.

    n_grams_flatten : dict
        A dictionary similar to `unique_n_grams`, but the values are flattened 
        lists of n-grams. This means that all n-grams of a particular length 
        are combined into a single list, making it easier to analyze the 
        frequency of occurrences.

    n_grams : dict
        A dictionary containing the original nested structure of n-grams. Each 
        key corresponds to the n-gram length, and the value is a list of lists, 
        where each inner list contains the n-grams for a specific sentence in 
        the corpus. This structure retains the context of where each n-gram 
        originated.

    train_data : list of list of tuples
        A list of tuples representing the padded n-grams generated from the 
        tokenized sentences. This is the data that can be used to train an 
        n-gram language model.

    vocab : list
        A list of unique words (tokens) found in the corpus, which serves as the 
        vocabulary for the n-gram model. This is useful for understanding the 
        diversity of the language used in the corpus.

    all_words : list
        A list of all words (tokens) from the corpus, including duplicates. This 
        provides insight into the overall word usage in the text.

    """    
    tokenized = [[w.lower() for w in word_tokenize(sent)] for sent in corpus]

    train_data, vocab = padded_everygram_pipeline(n, tokenized)
    all_words = list(vocab)
    train_data = [list(g) for g in list(train_data)]
    train_data = filter_invalid_ngrams(train_data)
    
    vocab = list(set(all_words))
    
    n_grams = {}
    n_grams_flatten = {}
    unique_n_grams = {}
    
    for j in range(n):
        n_grams[f'{j+1}-grams'] = get_Ngrams(train_data, j+1)
        
    for k in range(n):
        n_grams_flatten[f'{k+1}-grams'] = [item for sublist in n_grams[f'{k+1}-grams'] for item in sublist]

    for t in range(n):
        unique_n_grams[f'{t+1}-grams'] = list(set(n_grams_flatten[f'{t+1}-grams']))
            
    return unique_n_grams, n_grams_flatten, n_grams, train_data, vocab, all_words


uninq_n_grams, n_grams_flatten, n_grams,train_data, vocab, all_words = prepare_corupus(corpus, n=2)




## how to count grams or get n-grams 

In [2]:

from nltk.lm import NgramCounter

train_data_list = [list(s) for s in train_data] 


ng_counter = NgramCounter(train_data_list)


print("Unigram count for 'i':", ng_counter['i'])         # returns integer count of word on entire corpus 
print("Bigram continuations after ('i',):", sorted(ng_counter[('i',)].items()))
# get counts for full ngram 'i like' as indexing on a context then item:
print("C(i like) via counter:", ng_counter[['i']]['like'])   # equivalent: ng_counter[('i',)]['like']
# Access all bigrams (order=2) as a ConditionalFreqDist:

bigram_cfd = ng_counter[2]

for context in bigram_cfd.conditions():    
    for word in bigram_cfd[context]:
        print(f"{context} -> {word}: {bigram_cfd[context][word]}")
    

Unigram count for 'i': 5
Bigram continuations after ('i',): [('am', 2), ('do', 1), ('like', 2)]
C(i like) via counter: 2
('<s>',) -> i: 2
('<s>',) -> sam: 2
('<s>',) -> do: 1
('i',) -> am: 2
('i',) -> like: 2
('i',) -> do: 1
('am',) -> sam: 1
('am',) -> </s>: 1
('sam',) -> </s>: 3
('sam',) -> i: 2
('like',) -> sam: 2
('like',) -> </s>: 1
('do',) -> like: 1
('do',) -> i: 1


In [ ]:
# NEW FUNCTION:
def get_all_ngram_probabilities(model, all_ngrams_flattened):
    """
    Uses a trained MLE model to get probabilities for all n-grams.

    Args:
        model: The trained MLE language model.
        all_ngrams_flattened: A dictionary of flattened lists of n-grams,
                              keyed by their order (e.g., '1-grams', '2-grams').

    Returns:
        A list of dictionaries containing all n-gram probabilities for each order.
    """
    probabilities_list = [[] for _ in range(model.order)]
    for k in range(1, model.order + 1):
        prob_dict = {}
        ngram_list = all_ngrams_flattened[f'{k}-grams']
        for ngram in ngram_list:
            if k == 1:
                # Unigrams have no context
                prob = model.score(ngram[0])
            else:
                # Higher-order n-grams have a context
                context = ngram[:-1]
                word = ngram[-1]
                prob = model.score(word, context)
            prob_dict[ngram] = prob
        
        probabilities_list[k-1].append(prob_dict)
    return probabilities_list


corpus = TextFile_Corpus(path="./corpus.txt")
uninq_n_grams, n_grams_flatten, n_grams, train_data, vocab, all_words = prepare_corupus(corpus, n=2)

train_data_list = [list(s) for s in train_data]


ng_counter = NgramCounter(train_data_list)


# 4. Calculate probabilities using the new function
n = 2 # Let's calculate for unigrams, bigrams, and trigrams
lm = MLE(n)
lm.fit(train_data, vocab)

probabilities = get_all_ngram_probabilities(lm, n_grams_flatten)

probabilities[1]


[{('<s>', 'i'): 0.4,
  ('i', 'am'): 0.4,
  ('am', 'sam'): 0.5,
  ('sam', '</s>'): 0.6,
  ('<s>', 'sam'): 0.4,
  ('sam', 'i'): 0.4,
  ('am', '</s>'): 0.5,
  ('i', 'like'): 0.4,
  ('like', '</s>'): 0.3333333333333333,
  ('i', 'do'): 0.2,
  ('do', 'like'): 0.5,
  ('like', 'sam'): 0.6666666666666666,
  ('<s>', 'do'): 0.2,
  ('do', 'i'): 0.5}]

In [18]:
from nltk.lm import NgramCounter
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from collections import defaultdict
from itertools import permutations
from task1 import prepare_corupus, TextFile_Corpus

def addK_smoothing(ng_counter, max_order, k):
    """
    Calculates the Add-k smoothed probabilities for n-grams up to max_order.

    Args:
        ng_counter: An NgramCounter object containing n-gram counts.
        max_order: The maximum n-gram size to calculate probabilities for.
        k: The smoothing parameter (e.g., k=1 for Laplace smoothing).

    Returns:
        A list of dictionaries. The i-th dictionary contains the smoothed
        probabilities for n-grams of size i+1.
    """
    # Vocabulary size is needed for the denominator in the smoothing formula
    # We get it from the unigram counts.
    vocab_size = len(ng_counter[1].keys())
    
    probabilities_list = []
    
    # Iterate through each n-gram order from 1 to max_order
    for order in range(1, max_order + 1):
        prob_dict = {}
        
        if order == 1:
            unigram_counts = ng_counter[1]
            total_unigrams = unigram_counts.N()
            
            
            if total_unigrams > 0:
                for unigram, count in unigram_counts.items():
                    prob = (count ) / total_unigrams
                    prob_dict[unigram] = prob
        else:

            ngram_counts = ng_counter[order]
            context_counts = ng_counter[order - 1]

            all_contexts = list(context_counts.keys())
            all_words = list(ng_counter[1].keys())
            
            for context in all_contexts:
                context_count = context_counts.get(context, 0)
                denominator = context_count + k * vocab_size
                

                for word in all_words:
                    # Get the count of the full n-gram (context, word)
                    # The .get() method returns 0 if the n-gram is unseen.
                    ngram_count = ngram_counts[context].get(word, 0)
                    
                    prob = (ngram_count + k) / denominator
                    ngram = (context,) + (word,)
                    prob_dict[ngram] = prob
        
        probabilities_list.append(prob_dict)
        
    return probabilities_list

corpus = TextFile_Corpus(path="./corpus.txt")
uninq_n_grams, n_grams_flatten, n_grams, train_data, vocab, all_words = prepare_corupus(corpus, n=2)


train_data_list = [list(s) for s in train_data]

ng_counter = NgramCounter(train_data_list)

probabilities = addK_smoothing(ng_counter, 2, 1)

probabilities

[{'<s>': 0.18518518518518517,
  'i': 0.18518518518518517,
  'am': 0.07407407407407407,
  'sam': 0.18518518518518517,
  '</s>': 0.18518518518518517,
  'like': 0.1111111111111111,
  'do': 0.07407407407407407},
 {('<s>', '<s>'): 0.08333333333333333,
  ('<s>', 'i'): 0.08333333333333333,
  ('<s>', 'am'): 0.08333333333333333,
  ('<s>', 'sam'): 0.08333333333333333,
  ('<s>', '</s>'): 0.08333333333333333,
  ('<s>', 'like'): 0.08333333333333333,
  ('<s>', 'do'): 0.08333333333333333,
  ('i', '<s>'): 0.08333333333333333,
  ('i', 'i'): 0.08333333333333333,
  ('i', 'am'): 0.08333333333333333,
  ('i', 'sam'): 0.08333333333333333,
  ('i', '</s>'): 0.08333333333333333,
  ('i', 'like'): 0.08333333333333333,
  ('i', 'do'): 0.08333333333333333,
  ('am', '<s>'): 0.1111111111111111,
  ('am', 'i'): 0.1111111111111111,
  ('am', 'am'): 0.1111111111111111,
  ('am', 'sam'): 0.1111111111111111,
  ('am', '</s>'): 0.1111111111111111,
  ('am', 'like'): 0.1111111111111111,
  ('am', 'do'): 0.1111111111111111,
  ('sam

# task 4 
prepleixity 

In [24]:
import math
from nltk.lm import Lidstone
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import word_tokenize
from itertools import chain
from task1 import prepare_corupus, TextFile_Corpus, filter_invalid_ngrams


# 2. Prepare the corpus and train the Lidstone model
n = 2  # Bigram model
gamma = 1  # Lidstone smoothing parameter (0 < gamma <= 1)
# Note: Lidstone(gamma=1) is equivalent to Laplace (Add-1) smoothing.

corpus = TextFile_Corpus(path="./corpus.txt")

tokenized_sentences = [[w.lower() for w in word_tokenize(sent)] for sent in corpus]


train_data, vocab = padded_everygram_pipeline(n, tokenized_sentences)

n = 2
gamma= 1
# Instantiate the Lidstone model
lm = Lidstone(gamma, n)

# Fit the model with the prepared data
lm.fit(train_data, vocab)

# 3. Prepare the test sentences for perplexity calculation
test_sentence = "I like Sam"
test_sentence_oov = "I do not like Bob"

test_sentences = [test_sentence, test_sentence_oov]



tokenized_test_sentences = [[w.lower() for w in word_tokenize(sent)] for sent in test_sentences]
padded_test_data, _ = padded_everygram_pipeline(n, tokenized_test_sentences)
test_data = [list(g) for g in list(padded_test_data)]
test_data = filter_invalid_ngrams(test_data)


# 4. Use the built-in .perplexity() method
perplexity_score = lm.perplexity(list(test_data))

print(perplexity_score)


8.0


The value you obtained is correct. A perplexity score of 8 is the exact result for that test case with a Lidstone model with γ=1 (Laplace smoothing) trained on your corpus.

What Does a Perplexity of 8 Mean?
A perplexity score of 8 can be interpreted as follows:

On average, the model has the same level of uncertainty as if it were to choose uniformly from a set of 8 equally likely words at each point in the test sentence.

Since your vocabulary has 7 words (∣V∣=7), a perplexity of 8 is very high. It means the model is almost as "perplexed" as it would be if it had to choose from your entire vocabulary at random for every single word.

This high perplexity is expected because your test data includes an out-of-vocabulary word (Bob). Even though the smoothing gives the unseen bigrams a small non-zero probability, the model is still highly "surprised" by them. This shows that your model, while working correctly, is not a good fit for a test set that contains words it has never seen.